<a href="https://colab.research.google.com/github/ratul41907/Food-and-Nutrition-Based-RAG-Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn sentence-transformers transformers faiss-cpu pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 76.2 MB/s eta 0:00:00


In [2]:
import pandas as pd

In [11]:
from google.colab import drive


drive.mount('/content/drive')


Mounted at /content/drive


In [15]:
#library fot .parquet
!pip install pyarrow


In [14]:
import os
os.path.exists("/content/drive/MyDrive/Colab Notebooks/food_sample.parquet")


True

In [16]:
import pandas as pd

file_path = "/content/drive/MyDrive/Colab Notebooks/food_sample.parquet"
df = pd.read_parquet(file_path)

# quick inspection
print(df.shape)
print(df.columns)
df.head(3)


(35840, 110)
Index(['additives_n', 'additives_tags', 'allergens_tags', 'brands_tags',
       'brands', 'categories', 'categories_tags', 'categories_properties',
       'checkers_tags', 'ciqual_food_name_tags',
       ...
       'states_tags', 'stores_tags', 'stores', 'traces_tags', 'unique_scans_n',
       'unknown_ingredients_n', 'unknown_nutrients_tags', 'vitamins_tags',
       'with_non_nutritive_sweeteners', 'with_sweeteners'],
      dtype='object', length=110)


,additives_n,additives_tags,allergens_tags,brands_tags,brands,categories,categories_tags,categories_properties,checkers_tags,ciqual_food_name_tags,...,states_tags,stores_tags,stores,traces_tags,unique_scans_n,unknown_ingredients_n,unknown_nutrients_tags,vitamins_tags,with_non_nutritive_sweeteners,with_sweeteners
0,NaN,None,[en:nuts],[xx:bovetti],Bovetti,"Petit-déjeuners,Produits à tartiner,Produits à...","[en:breakfasts, en:spreads, en:sweet-spreads, ...","{'agribalyse_food_code': 31032.0, 'agribalyse_...",[],[chocolate-spread-with-hazelnuts],...,"[en:to-be-completed, en:nutrition-facts-comple...",[],,[],1.0,NaN,[],[],NaN,NaN
1,0.0,[],[],[lagg-s],Lagg's,null,[en:null],"{'agribalyse_food_code': None, 'agribalyse_pro...",[],[unknown],...,"[en:to-be-completed, en:nutrition-facts-comple...",None,<NA>,[],1.0,0.0,[],[],NaN,NaN
2,0.0,[],[],[lagg-s],Lagg's,"Plant-based foods and beverages, Beverages, Ho...","[en:plant-based-foods-and-beverages, en:bevera...","{'agribalyse_food_code': 18020.0, 'agribalyse_...",[],[unknown],...,"[en:to-be-completed, en:nutrition-facts-comple...",None,<NA>,[],NaN,0.0,[],[],NaN,NaN


In [18]:
df.columns.tolist()


['additives_n',
 'additives_tags',
 'allergens_tags',
 'brands_tags',
 'brands',
 'categories',
 'categories_tags',
 'categories_properties',
 'checkers_tags',
 'ciqual_food_name_tags',
 'cities_tags',
 'code',
 'compared_to_category',
 'complete',
 'completeness',
 'correctors_tags',
 'countries_tags',
 'created_t',
 'creator',
 'data_quality_errors_tags',
 'data_quality_info_tags',
 'data_quality_warnings_tags',
 'data_sources_tags',
 'ecoscore_data',
 'ecoscore_grade',
 'ecoscore_score',
 'ecoscore_tags',
 'editors',
 'emb_codes_tags',
 'emb_codes',
 'entry_dates_tags',
 'food_groups_tags',
 'generic_name',
 'images',
 'informers_tags',
 'ingredients_analysis_tags',
 'ingredients_from_palm_oil_n',
 'ingredients_n',
 'ingredients_original_tags',
 'ingredients_percent_analysis',
 'ingredients_tags',
 'ingredients_text',
 'ingredients_with_specified_percent_n',
 'ingredients_with_unspecified_percent_n',
 'ingredients_without_ciqual_codes_n',
 'ingredients_without_ciqual_codes',
 'ingre

In [20]:
import pandas as pd
import json

# Reload the parquet file (optional safety)
file_path = "/content/drive/MyDrive/Colab Notebooks/food_sample.parquet"
df = pd.read_parquet(file_path)

# Function to safely extract numeric nutrients
def safe_get(d, key):
    try:
        return d.get(key, 0) if isinstance(d, dict) else 0
    except:
        return 0

# Build a simplified DataFrame
food_df = pd.DataFrame({
    "product_name": df["product_name"],
    "ingredients_text": df.get("ingredients_text", ""),
    "category": df.get("categories", ""),
    "brand": df.get("brands", ""),
    "nutri_grade": df.get("nutriscore_grade", ""),
    "energy_kcal": df["nutriments"].apply(lambda x: safe_get(x, "energy-kcal_100g")),
    "proteins": df["nutriments"].apply(lambda x: safe_get(x, "proteins_100g")),
    "fat": df["nutriments"].apply(lambda x: safe_get(x, "fat_100g")),
    "sugar": df["nutriments"].apply(lambda x: safe_get(x, "sugars_100g")),
})

# Drop incomplete rows and fill missing text
food_df = food_df.dropna(subset=["product_name"]).fillna("")
food_df["ingredients_text"] = food_df["ingredients_text"].astype(str).str[:300]


food_df.head(3)


,product_name,ingredients_text,category,brand,nutri_grade,energy_kcal,proteins,fat,sugar
0,"[{'lang': 'main', 'text': 'Véritable pâte à ta...",[],"Petit-déjeuners,Produits à tartiner,Produits à...",Bovetti,e,0,0,0,0
1,"[{'lang': 'main', 'text': 'Chamomile Herbal Te...","[{'lang': 'main', 'text': 'CHAMOMILE FLOWERS.'...",null,Lagg's,unknown,0,0,0,0
2,"[{'lang': 'main', 'text': 'Lagg's, herbal tea,...","[{'lang': 'main', 'text': 'Peppermint.'}\n {'l...","Plant-based foods and beverages, Beverages, Ho...",Lagg's,unknown,0,0,0,0


In [21]:
def make_text(row):
    return (
        f"{row['product_name']} ({row['brand']}): "
        f"{row['energy_kcal']} kcal per 100 g | Protein {row['proteins']} g | "
        f"Fat {row['fat']} g | Sugar {row['sugar']} g. "
        f"Category: {row['category']}. "
        f"Nutri-Score: {row['nutri_grade']}. "
        f"Ingredients: {row['ingredients_text']}"
    )

food_df["text"] = food_df.apply(make_text, axis=1)

# Keep only id + text for RAG retriever
food_knowledge = food_df[["text"]].reset_index().rename(columns={"index": "id"})

print("✅ Text generation complete.")
food_knowledge.head(3)


✅ Text generation complete.


,id,text
0,0,"[{'lang': 'main', 'text': 'Véritable pâte à ta..."
1,1,"[{'lang': 'main', 'text': 'Chamomile Herbal Te..."
2,2,"[{'lang': 'main', 'text': ""Lagg's, herbal tea,..."


In [22]:
output_path = "/content/drive/MyDrive/Colab Notebooks/food_knowledge.json"

records = food_knowledge.to_dict(orient="records")
with open(output_path, "w") as f:
    json.dump(records, f, indent=2)

print(f"✅ Saved knowledge base: {output_path}")
print("Total records:", len(records))


✅ Saved knowledge base: /content/drive/MyDrive/Colab Notebooks/food_knowledge.json
Total records: 35840
